# Attention

## Self-Attention

$$
\mathrm{Attention}(Q, K, V) = \mathrm{Softmax}!\left( \frac{QK^{\mathsf T}}{\sqrt{d_k}} + M \right)V
$$

**Self-Attention（自注意力）** 是大型语言模型（LLM）中的核心机制。
它允许序列中每个 token 与同一序列中的其他 token 建立依赖关系。
以下以输入文本 “a fluffy blue creature” 为例说明计算过程，并标注各个张量的维度（含 batch size）。

---

### 计算 Q、K、V 矩阵

输入张量：

$$
X \in \mathbb{R}^{B \times n \times d_{\text{model}}}
$$

其中：

* $B$：batch size（批量大小）
* $n$：序列长度（例如 4）
* $d_{\text{model}}$：模型的嵌入维度（例如 12288）

对输入进行线性变换， 作用为将输入映射到新的张量：

$$
Q = XW_Q, \quad K = XW_K, \quad V = XW_V
$$

权重矩阵和结果的维度为：

$$
W_Q, W_K, W_V \in \mathbb{R}^{d_{\text{model}} \times d_k}, \qquad Q, K, V \in \mathbb{R}^{B \times n \times d_k}
$$

通常 $d_k \ll d_{\text{model}}$，以降低计算复杂度。

语义示例：“creature” 的 $Q$ 向量偏向寻找形容词，“fluffy” 的 $K$ 向量则偏向标识自身为形容词。

---

### 计算注意力分数（Attention Score）

对每个 batch 独立计算注意力分数：

$$
S[b] = \frac{Q[b] K[b]^{\mathsf T}}{\sqrt{d_k}}, \quad b = 1, \dots, B
$$

整体张量：

$$
S \in \mathbb{R}^{B \times n \times n}
$$

每个元素 $S_{b, i, j}$ 表示第 $b$ 个样本中第 $i$ 个 token 的 query 与第 $j$ 个 token 的 key 的匹配程度。

除以 $\sqrt{d_k}$ 可以防止数值过大，避免 Softmax 输出饱和导致梯度消失。

---

### Softmax 归一化

对每个 batch 的每个 query（第 $i$ 行）在 key 维度上执行 Softmax：

$$
A[b] = \mathrm{Softmax}(S[b]), \quad A \in \mathbb{R}^{B \times n \times n}
$$

保证每行的权重和为 1：

$$
\sum_{j=1}^{n} A_{b, i, j} = 1, \quad \forall b, i
$$

示例：
“creature” 的注意力权重中，“fluffy” 和 “blue” 的概率较高，“a” 的权重较低。

---

### 加权求和得到上下文表示

利用注意力权重加权求和 Value 向量：

$$
O[b] = A[b] V[b], \quad b = 1, \dots, B
$$

输出张量：

$$
O \in \mathbb{R}^{B \times n \times d_k}
$$

每个 $O_{b, i, :}$ 是第 $b$ 个样本中第 $i$ 个 token 的上下文表示，即对所有 $V_{b, j, :}$ 的加权和（权重为 $A_{b, i, j}$）。

示例：“creature” 的输出向量融合了 “fluffy”（毛茸茸）和 “blue”（蓝色）的语义，从“通用生物”变为“毛茸茸的蓝色生物”。

In [1]:
import torch
import torch.nn as nn

/opt/anaconda3/envs/handonLLM/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
class SelfAttention(nn.Module):
    def __init__(self, d_model, d_k, d_v) -> None:
        super().__init__()
        self.d_model = d_model
        self.d_k = d_k
        self.d_v = d_v

        # 将d_model映射为d_k/d_v
        self.W_q = nn.Linear(self.d_model, self.d_k)
        self.W_k = nn.Linear(self.d_model, self.d_k)
        self.W_v = nn.Linear(self.d_model, self.d_v)

        # 需将d_v映射回d_model
        self.output_proj = nn.Linear(self.d_v, self.d_model)

    def forward(self, x):
        # x.shape = (batch_size, seq_len, d_model)

        # _ = x @ W
        # (batch_size, seq_len, d_model) ->  (batch_size, seq_len, d_k/d_v)
        Q = self.W_q(x) 
        K = self.W_k(x) 
        V = self.W_v(x) 

        # (batch_size, seq_len, d_k/d_v)->(batch_size, d_k/d_v, seq_len)
        # (batch_size, seq_len, d_k/d_v) @ (batch_size, d_k/d_v, seq_len)
        # 衡量“当前Token的Q”与“其他所有Token的K”的匹配度（关联度） 
        attention_score = (torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float32, device=x.device)))

        # (batch_size, seq_len, seq_len) 需要按列softmax -> dim=-1
        attention_weight = torch.softmax(attention_score, dim=-1)

        # 通过attention_weight计算出Value
        # (batch_size, seq_len, seq_len) @ (batch_size, seq_len, d_v)
        output = attention_weight @ V

        # 通过输出线性层映射回d_model
        output = self.output_proj(output)
        return output

In [3]:
x = torch.rand(3, 2, 4)
net = SelfAttention(d_model=4, d_k=5, d_v=3)
print(net(x))

tensor([[[ 0.2391, -0.0947,  0.2341, -0.3469],
         [ 0.2393, -0.0945,  0.2352, -0.3482]],

        [[ 0.2411, -0.0467,  0.3459, -0.4229],
         [ 0.2415, -0.0468,  0.3464, -0.4239]],

        [[ 0.2192, -0.1058,  0.2581, -0.3823],
         [ 0.2189, -0.1054,  0.2585, -0.3822]]], grad_fn=<ViewBackward0>)


## Self-Attention 优化
### Masking（掩码）防止信息泄露

在自回归任务中，模型不应看到未来的 token。
因此在 Softmax 前添加掩码矩阵 $M$：

$$
A = \mathrm{Softmax}\left( \frac{QK^{\mathsf{T}}}{\sqrt{d_k}} + M \right)
$$

其中掩码定义为：

$$
M_{i,j} = 
\begin{cases} 
0, & j \le i, \\
-\infty, & j > i.
\end{cases}
$$

Softmax 后，$M_{i,j} = -\infty$ 的位置权重自动变为 0。

此外，还可使用 **Padding Mask**，对 `[PAD]` 等填充 token 的注意力分数设为 $-\infty$，避免无效位置干扰。

---

### Dropout（自注意力中的随机丢弃）

**作用**  
- 防止过拟合，增强模型泛化能力；
- 在训练中随机丢弃部分注意力连接。

**在注意力权重上的 Dropout**  

在 Softmax 后对注意力矩阵应用 Dropout：

$$
A' = \mathrm{Dropout}(A; p), \qquad A \in \mathbb{R}^{B \times h \times n \times n}
$$

$$
A'_{b,i,j} =
\begin{cases}
\dfrac{A_{b,i,j}}{1-p}, & \text{若位置 }(b,i,j)\text{ 未被丢弃}, \\
0, & \text{若位置 }(b,i,j)\text{ 被丢弃}.
\end{cases}
$$

Dropout 生成与 $A$ 同形状的掩码 $M_{\text{drop}} \in \{0,1\}^{B \times h \times n \times n}$，并对未丢弃部分按 $1/(1-p)$ 放缩（**Inverted Dropout**）。

输出再与 $V$ 相乘：

$$
O = A'V, \qquad O \in \mathbb{R}^{B \times h \times n \times d_{\text{head}}}
$$

**训练与推理阶段的区别**  
- 训练：随机丢弃部分注意力连接，保留部分按 $1/(1-p)$ 放缩 
- 推理：禁用 Dropout，使用完整注意力权重矩阵

In [12]:
class SelfAttentionV2(nn.Module):
    def __init__(self, d_model, d_k, d_v, dropout_rate=0.1) -> None:
        super().__init__()
        # 输入的维度
        self.d_model = d_model
        self.d_k = d_k
        self.d_v = d_v
        self.dropout_rate = dropout_rate
        # 将d_model映射为d_k/d_v
        self.W_q = nn.Linear(self.d_model, self.d_k)
        self.W_k = nn.Linear(self.d_model, self.d_k)
        self.W_v = nn.Linear(self.d_model, self.d_v)

        # Dropout层
        self.dropout = nn.Dropout(self.dropout_rate)

        # 需将d_v映射回d_model
        self.output_proj = nn.Linear(self.d_v, self.d_model)

    def forward(self, x, attention_mask=None):
        # x.shape = (batch_size, seq_len, d_model)

        # _ = x @ W
        # (batch_size, seq_len, d_model) ->  (batch_size, seq_len, d_k/d_v)
        Q = self.W_q(x) 
        K = self.W_k(x) 
        V = self.W_v(x) 

        # (batch_size, seq_len, d_k/d_v)->(batch_size, d_k/d_v, seq_len)
        # (batch_size, seq_len, d_k/d_v) @ (batch_size, d_k/d_v, seq_len)
        # 衡量“当前Token的Q”与“其他所有Token的K”的匹配度（关联度） 
        attention_score = (torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float32, device=x.device)))
        # mask
        # attention_mask shape is (batch, seq)
        if attention_mask is not None:
            attention_score = attention_score.masked_fill(attention_mask==0,  float("-inf"))

        # (batch_size, seq_len, seq_len) 需要按列softmax -> dim=-1
        attention_weight = torch.softmax(attention_score, dim=-1)

        # 传入矩阵，dropout
        attention_weight = self.dropout(attention_weight)

        # 通过attention_weight计算出Value
        # (batch_size, seq_len, seq_len) @ (batch_size, seq_len, d_v)
        output = attention_weight @ V

        # 通过输出线性层映射回d_model
        output = self.output_proj(output)
        return output

In [14]:
X = torch.rand(3, 4, 2)
b = torch.tensor(
    [
        [1, 1, 1, 0],
        [1, 1, 0, 0],
        [1, 0, 0, 0],
    ]
)
mask = b.unsqueeze(dim=1).repeat(1, 4, 1)

net = SelfAttentionV2(2, 2, 2)
print(X)
net(X, mask)

tensor([[[0.2835, 0.5654],
         [0.7871, 0.9491],
         [0.5740, 0.2463],
         [0.4636, 0.5301]],

        [[0.9686, 0.5447],
         [0.3322, 0.1100],
         [0.5974, 0.2093],
         [0.6519, 0.1966]],

        [[0.0341, 0.8704],
         [0.3002, 0.9153],
         [0.5888, 0.3706],
         [0.0245, 0.0150]]])


tensor([[[ 0.4857, -1.0564],
         [ 0.4382, -0.8936],
         [ 0.4859, -1.0542],
         [ 0.4858, -1.0556]],

        [[ 0.5426, -1.0690],
         [ 0.5410, -1.0720],
         [ 0.5422, -1.0698],
         [ 0.5426, -1.0690]],

        [[ 0.3721, -0.3863],
         [ 0.4744, -0.8330],
         [ 0.4744, -0.8330],
         [ 0.4744, -0.8330]]], grad_fn=<ViewBackward0>)

## 多头注意力（Multi-Head Attention）

多头注意力机制（**Multi-Head Attention, MHA**）最早由 *Vaswani et al.* 在论文 **Attention Is All You Need (2017)** 中提出，用于增强模型捕捉多种依赖关系的能力。

单头注意力（Single-Head Attention）只能学习一种类型的上下文关系（例如形容词与名词之间的依赖），  
而多头注意力通过并行多个独立的注意力头，使模型能够同时学习语法、语义以及长距离指代等不同类型的依赖。

---

### 数学定义

多头注意力的整体定义为：

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h) W^O
$$

其中每个注意力头的计算为：

$$
\text{head}_i = \text{Attention}(Q W_i^Q, K W_i^K, V W_i^V)
$$

单个注意力函数（Scaled Dot-Product Attention）的形式为：

$$
\text{Attention}(Q, K, V) = \text{Softmax}\left(\frac{Q K^\top}{\sqrt{d_k}}\right)V
$$

---

### 矩阵维度关系

设：

- 批量大小为 **B**，序列长度为 **n**；
- 模型隐层总维度为 **d_model**；
- 注意力头的数量为 **h**；
- 每个注意力头的特征维度为  
  $d_k = d_v = d_{\text{model}} / h$。

则各矩阵的维度关系如下：

1. **输入矩阵：**

   $$
   Q, K, V \in \mathbb{R}^{B \times n \times d_{\text{model}}}
   $$

2. **每个注意力头的线性映射参数：**

   $$
   W_i^Q, W_i^K, W_i^V \in \mathbb{R}^{d_{\text{model}} \times d_k}
   $$

   投影后：

   $$
   Q_i = Q W_i^Q,\quad K_i = K W_i^K,\quad V_i = V W_i^V
   $$

   此时：

   $$
   Q_i, K_i, V_i \in \mathbb{R}^{B \times n \times d_k}
   $$

3. **每个头的输出：**

   $$
   O_i = \text{Softmax}\left(\frac{Q_i K_i^\top}{\sqrt{d_k}}\right)V_i
   $$

   输出维度：

   $$
   O_i \in \mathbb{R}^{B \times n \times d_v}
   $$

4. **拼接所有头的结果：**

   $$
   O_{\text{concat}} = [O_1; O_2; \dots; O_h] \in \mathbb{R}^{B \times n \times (h \cdot d_v)} = \mathbb{R}^{B \times n \times d_{\text{model}}}
   $$

5. **最终线性变换：**

   - 线性映射矩阵：  
     $$
     W^O \in \mathbb{R}^{(h \cdot d_v) \times d_{\text{model}}}
     $$
   - 输出为：  
     $$
     O_{\text{final}} = O_{\text{concat}} W^O \in \mathbb{R}^{B \times n \times d_{\text{model}}}
     $$

---

### 计算流程

1. **分头线性映射：**  
   将输入的 \( Q, K, V \) 分别投影为 \( h \) 个子空间：

   $$
   Q_i = Q W_i^Q, \quad K_i = K W_i^K, \quad V_i = V W_i^V
   $$

2. **各头独立计算注意力：**  
   对每个头并行计算缩放点积注意力：

   $$
   O_i = \text{Softmax}\left(\frac{Q_i K_i^\top}{\sqrt{d_k}}\right)V_i
   $$

3. **拼接与线性变换：**  
   拼接所有头的输出，再经线性层恢复至原始维度：

   $$
   O_{\text{final}} = [O_1; O_2; \dots; O_h] W^O
   $$

---

### 多头注意力的优势

1. **多层次依赖建模：**  
   不同注意力头可捕捉不同类型的语言特征：
   - 某些头学习局部语法结构（如冠词–名词搭配）；
   - 另一些头学习长距离依赖（如代词与前文实体的对应）。

2. **增强鲁棒性：**  
   多个头的结果互补，降低单一注意力机制的偏差风险。

3. **可解释性：**  
   实证研究发现，不同注意力头在语法、语义和实体关系等方面表现出特定的专门化模式。


In [15]:
import torch
import torch.nn as nn
import math

class MultiHeadAttention(nn.Module):
    # d_model == d_k == d_v
    def __init__(self, d_model, head_num, dropout_rate=0.1) -> None:
        super().__init__()
        # 输入的维度
        self.d_model = d_model
        self.dropout_rate = dropout_rate
        # head_num and head_dim
        self.head_num = head_num
        self.head_dim = self.d_model // head_num
        # 将d_model映射为d_k/d_v
        self.W_q = nn.Linear(self.d_model, self.d_model)
        self.W_k = nn.Linear(self.d_model, self.d_model)
        self.W_v = nn.Linear(self.d_model, self.d_model)

        # Dropout层
        self.dropout = nn.Dropout(self.dropout_rate)

        # 需将d_v映射回d_model
        self.output_proj = nn.Linear(self.d_model, self.d_model)

        # 预先计算缩放因子，避免每次forward都创建tensor
        self.scale = math.sqrt(self.head_dim)

    def forward(self, query, key=None, value=None, attention_mask=None):
        # 兼容 self-attention 和 cross-attention
        if key is None:
            key = query
        if value is None:
            value = query

        # x.shape = (batch_size, seq_len, d_model)
        batch_size, seq_len, _ = query.size()

        # _ = x @ W
        # (batch_size, seq_len, d_model) ->  (batch_size, seq_len, d_k/d_v)
        Q = self.W_q(query) 
        K = self.W_k(key) 
        V = self.W_v(value) 

        # shape: （batch_size, seq_len, d_model=head_num * head_dim) ->（batch_size, num_head, seq_len, head_dim)
        q_state = Q.view(batch_size, seq_len, self.head_num, self.head_dim).transpose(1, 2)
        k_state = K.view(batch_size, K.size(1), self.head_num, self.head_dim).transpose(1, 2)
        v_state = V.view(batch_size, V.size(1), self.head_num, self.head_dim).transpose(1, 2)

        # (batch_size, head_num, seq_len, head_dim)->(batch_size, head_num, head_dim, seq_len)
        # (batch_size, head_num, seq_len, head_dim)->(batch_size, head_num, seq_len, seq_len)
        attention_score = torch.matmul(q_state, k_state.transpose(-2, -1)) / self.scale

        # mask
        if attention_mask is not None:
            # 保证mask维度正确 (batch_size, 1, 1, seq_len)
            if attention_mask.dim() == 2:
                attention_mask = attention_mask[:, None, None, :]
            attention_score = attention_score.masked_fill(attention_mask == 0, float("-inf"))

        # (batch_size, head_num, seq_len, seq_len) 需要按列softmax -> dim=-1
        attention_weight = torch.softmax(attention_score, dim=-1)

        # 传入矩阵，dropout
        attention_weight = self.dropout(attention_weight)

        # 通过attention_weight计算出Value
        # (batch_size, head_num, seq_len, seq_len) @ (batch_size, head_num, seq_len, d_v)
        output = torch.matmul(attention_weight, v_state)

        # 多头聚合
        # 重新变成 (batch, seq_len, num_head, head_dim)
        # 这里的 contiguous() 是相当于返回一个连续内存的 tensor，一般用了 permute/tranpose 都要这么操作
        output = output.transpose(1, 2).contiguous()
        output = output.view(batch_size, seq_len, -1)

        # 通过输出线性层映射回d_model
        output = self.output_proj(output)

        # 返回输出与注意力权重（方便可视化或调试）
        return output, attention_weight

In [21]:
attention_mask = (
    torch.tensor(
        [
            [0, 1],
            [0, 0],
            [1, 0],
        ]
    )
    .unsqueeze(1)
    .unsqueeze(2)
    .expand(3, 8, 2, 2)
)

x = torch.rand(3, 2, 128)
net = MultiHeadAttention(128, 8)
net(x, attention_mask=attention_mask)[0].shape

torch.Size([3, 2, 128])